In [2]:
import pandas as pd
from iexfinance.stocks import get_historical_data
from iexfinance.stocks import get_historical_intraday
from iexfinance.stocks import Stock
from datetime import date
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

In [3]:
def create_dict(stocklist): 
    ''' Creates dictionary of a list of stocks
    
        :param stocklist: the list of stocks that you want to gather data for
        :type stocklist: List of Strings
        :returns: empty dictionary for each stock
        :rtype: dictionary
        
    '''
    stockdict = {}
    for s in stocklist:
        stockdict[s] = {}
    return stockdict

def data(stockdict, start_date, weeks):
    ''' Gathers Minute data for each stock for each day within a span of x weeks\
        and compiles into dataframe to save into dictionary
        
        :param stockdict: a dictionary that has stock tickers as keys
        :type stockdict: dictionary
        :param start_date: start date of when to collect intraday data (90 day lookback period)
        :type start_date: String
        :param weeks: how many weeks ahead do you want to get intraday data for starting from start_date
        :type weeks: int
        :returns: dictionary with basic market metrics
        :rtype: dictionary
        
    '''
    t = "pk_80254a25b98240389c6c03f7bae9c68a"
    global date
    start = pd.to_datetime(start_date)
    weekday = []
    for i in range(7*weeks):
        d = start + timedelta(days=i)
        if d.weekday() < 5:            
            weekday.append(d)
    for s in stockdict.keys():
        df = pd.DataFrame()
        for d in weekday:
            date = d
            intra = get_historical_intraday(s, date, output_format='pandas', token = t)
            frames = [df, intra]
            df = pd.concat(frames)
        df = df[['marketHigh','marketLow','marketAverage','marketVolume','marketOpen','marketClose']]
        df = df.sort_index()
        df = df.dropna()
        data = df
        stockdict[s]['data'] = data
        stockdict[s]['open'] = data['marketOpen']
        stockdict[s]['high'] = data['marketHigh']
        stockdict[s]['low'] = data['marketLow']
        stockdict[s]['close'] = data['marketClose']
        stockdict[s]['volume'] = data['marketVolume']
        data.to_csv(s + '.csv')
    return stockdict   